# Abstract

In this project, the environment of the reinforcement learning project is built in a Grid World. There are two environments based on Grid World. A Deterministic environment and a Stochastic Environment. It as been assumed that the structure of the environment is known and hence Value Iteration is used to solve the control problem of the environment.

# Introduction

## Components in Reinforcement Learning Environment

Reinforcement Learing mainly contains two entities
1. Agent
2. Environment
3. Reward
4. State

 <img src="./files/components.png" style="width: 400px">

**1. Agent**: The agent is the entity that learns and takes actions inside the environment. The main goal of the agent is to maximize the reward it obtains of a period of time

**2. Environment**: The Environment is the entity that gives rewards to the agents and also gives the state in which the Agent ends up in after the Agent interacts with the environment by taking action. There are Reinforcement Problems wherein the strucutre and behaviour of the Environment are known (Model_Based) and some Evnironments where the behaviour of the environment is unknown (Model-Free)

**3. Reward**: The reward is a single integer value returned to the agent by the environment as result of the agents interaction (actions) with the environment. The agents aims to increase these rewards in the long run.

**4. State**: The Environment at each instant of time is called a state. The agents moves from one state to another by performing actions

## Types of Environments

In this project. Two kinds of environments are solved. 
1. Deterministic: In this kind of environment, given the current state of the environment and the action of the agent the next state to which the agent transitions is fixed. Also every time the agents takes an action from a particular state, the immediate reward recieved will be same all the time.

2. Stochastic Environment: In this kind of an environment, given current state and the action of the agent taken fron that state, the state in which the agent ends up is not certain. There may be more than one possible states the agents might end up after taking a particular action. Also, in a stochastic environment every time the agents takes a particular action from a particular state, the immediate reward received might not be the same since there are more than one possible state the agent ends up in

## Markov Decision Process

Markov Decision Process (MPD) is a Markov Reward Process with decisions (because actions are involved). In this each process is a Markov Process. MDP tuple is **(S, A, P, R, 𝛾)** 

1. **S**: State Space
2. **A**: A set of finite actions that can be taken by the agent
3. **P**: State Transition Probabilities Function
4. **R**: Reward Function
5. **𝛾**: Discount Factor

The follwing formula denote P and R:

<img src="./files/trans_prob_function.png" width="400px">

 <img src="./files/reward_function.png" style="width: 400px;">

# Problem Setup

In this project, two kinds of environments are used to illustrate the value iteration to solve the control problem

**Deterministic Grid World**: The agent (car) in this environment moves through a 3x4 grid. The final terminal (winning) state is S(0,3) which gives a reward of +10 and there is a state just below the final state, which is bad state to be in since there is a raging fire in the state and going there would give a reward of -5. There is also another terminal state (1,1) which is a game over state (if the agent goes into that state, it is stuck).

**Stochastic Grid World**: This environment is a slightly modified from the Deterministic Grid World, wherein we assume the car faulty steering wheele and hence, when aciton "right" or "left" is taken, there is 80% chance that bheaves accordinly, and there is chance of 15% that the car goes to a state that is above and 5% chance the car goes to a state that is below, similarly when action "up" and "down" is taken the car goes right with a 17% chance and goes left with 3% chance

## Dynamic Programming (Value Iteration)

In this project, Dynamic Programming (Value Iteration) is used to find the optimal policy. This method assumes full knowledge of the MDP (knowledge of how to the environment works) it is a model-based method of solving the environment

This planning method is based on the Optimality Theorm 


**A policy 𝜋(a|s) achieves the optimal value from state s, i.e. V𝜋(s) = V\*(s), if and only if, for any state s′ reachable from s, 𝜋 achieves the optimal value from state s′, V𝜋(s′) = V\*(s′).**

If the solution to V*(s') is known, then then V*(s) can be found by one step look ahead as shown below

 <img src="./files/bellman_optimality_equation.png" style="width: 400px;">

Further, this bellman optimality eqation can be iteratively applied until the values converge to give an optimal value for each state of the environment. After this, the agent can act greedily (to select the action that reaches the state with the highest value at each time step to reach the winning state.

<img src="./files/bellman_iterative.png" width="400px">

** Please note: There are utility functions and other dependencies that work only when cells are run in a sequential order. While evaluating to reproduce the results kindly run the cells seqeuntially**

# Deterministic Environment

### Utility Functions for visualization of results 

In [262]:
def print_values(V, g):
    for i in range(g.width):
        print("---------------------------")
        for j in range(g.height):
            v = V.get((i,j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="") # -ve sign takes up an extra space
        print("")

def print_policy(P, g):
    for i in range(g.width):
        print("\n---------------------------")
        for j in range(g.height):
            a = P.get((i,j), ' ')
            print("  %s  |" % a, end="")
    print("")

### Actual Implementation

In [263]:
class Grid:
    def __init__(self, width, height, start):
        self.width = width
        self.height = height
        self.row = start[0]
        self.col = start[1]
        
    def step(self, action):
        if action in self.actions[(self.row, self.col)]:
            if action == "L":
                self.row -= 1
            elif action == "R":
                self.row += 1
            elif action == "U":
                self.col += 1
            elif action == "D":
                self.col -= 1
            return self.rewards.get((self.row, self.col), 0)
            
            
    def reset(self, rewards, actions):
        self.actions = actions
        self.rewards = rewards
        
    def find_next_state_reward(self, action):
        agent_pos_row = self.row
        agent_pos_col = self.col
        if action in self.actions[(agent_pos_row, agent_pos_col)]:
            if action == "U":
                if self.check_boundary_and_invalid(agent_pos_row-1, agent_pos_col):
                    agent_pos_row -= 1
                else:
                    pass
            elif action == "D":
                if self.check_boundary_and_invalid(agent_pos_row+1, agent_pos_col ):
                    agent_pos_row += 1
                else:
                    pass
            elif action == "R":
                if self.check_boundary_and_invalid(agent_pos_row, agent_pos_col+1):
                    agent_pos_col += 1
                else:
                    pass
            elif action == "L":
                if self.check_boundary_and_invalid(agent_pos_row, agent_pos_col-1):
                    agent_pos_col -= 1 
                else:
                    pass
        else:
            pass
        
        reward = self.rewards.get((agent_pos_row, agent_pos_col), 0)
        return ((agent_pos_row, agent_pos_col), reward)
    
    def check_boundary_and_invalid(self,row, col):
        if row >=0 and row <= self.height:
                if col >=0 and col <= self.width:
                    return True
        return False
        
    def non_terminal_states(self):
        return self.actions.keys()

    def set_agent_position(self, s):
        self.row = s[0]
        self.col = s[1]
        
    
    def trans_probs(self, action):
        # returns a transition probablity, reward, state tuple
        trans_probs = []
        state, reward = self.find_next_state_reward(action)
        trans_probs.append((1, reward, state))
        return trans_probs
    
    def all_states(self):
        return set(self.actions.keys()) | set(self.rewards.keys())


The above class is used to build the Deterministic Environment Grid

In [264]:
class Agent():
    def __init__(self, grid, discount):
        self.environment = grid
        self.state_value = {}
        self.action_value = {}
        self.valid_actions = ("U","D","L","R") # (Up, Down, Left, Right)
        self.discount = discount
        
        for s in grid.all_states():
            self.state_value[s] = 0
        
    
    def find_value_action(self, s):
        #Function to find the optimal state value of the current state based on the state value of the next state from previous iteration
        best_action = "GO"
        best_value = float('-inf')
        self.environment.set_agent_position(s)
        legal_actions = self.environment.actions[s]
        for action in legal_actions:
            transitions = self.environment.trans_probs(action)
            expected_value = 0
            expected_reward = 0
            for (prob, immediate_reward, next_state) in transitions:
                expected_reward += prob * immediate_reward
                expected_value += prob * self.state_value[next_state]
            current_value = expected_reward + discount * expected_value
            if current_value > best_value:
                best_value = current_value
                best_action = action
        return best_action, best_value

    def find_v(self):
        states = self.environment.all_states()
        for s in self.environment.non_terminal_states():
            previous_state_value = self.state_value[s]
            _, new_state_value = self.find_value_action(s)
            self.state_value[s] = new_state_value
        return self.state_value

    def initialize_random_policy(self):
        policy = {}
        for s in self.environment.non_terminal_states():
            policy[s] = np.random.choice(self.valid_actions)
        return policy

    def calculate_greedy_policy(self,V):
        policy = self.initialize_random_policy()
        for s in policy.keys():
            self.environment.set_agent_position(s)
        # consider actions to find the best current action
            best_action, _ = self.find_value_action(s)
            policy[s] = best_action
        return policy
    
    def value_iteration(self, iterations):
        print("Rewards:")
        print_values(self.environment.rewards, self.environment)
        for i in range(iterations):
            print("\n------- Iteration {}-----------\n".format(i))
            values = self.find_v()
            print("Values:")
            print_values(values, self.environment)
            
        greedy_policy = self.calculate_greedy_policy(values)
        print("\npolicy:")
        print_policy(greedy_policy, self.environment)

Since it is assumed the full knowledge of MDP for value iteration, the actions are given expilictly while creating the grid object

In [265]:
def grid():
    g = Grid(3, 4, (2, 0))
    rewards = {(0, 3): 10, 
               (1, 3): -5,}
    
    actions = { (0, 0): ('D', 'R'), (0, 1): ('L', 'R'), (0, 2): ('L', 'D', 'R'), (1, 0): ('U', 'D'), (1,1): (),
    (1, 2): ('U', 'D', 'R'), (1,3) : ("L", "D"),  (2, 0): ('U', 'R'), (2, 1): ('L', 'R'),(2, 2): ('L', 'R', 'U'),(2, 3): ('L', 'U'),}
    g.reset(rewards, actions)
    return g

In [266]:
g = grid()
a = Agent(g, 0.9)

### Results of Deterministic Environment

In [267]:
a.value_iteration(4)

Rewards:
---------------------------
 0.00| 0.00| 0.00| 10.00|
---------------------------
 0.00| 0.00| 0.00|-5.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|

------- Iteration 0-----------

Values:
---------------------------
 0.00| 0.00| 10.00| 0.00|
---------------------------
 0.00|-inf| 5.00| 2.50|
---------------------------
 0.00| 0.00| 2.50| 1.25|

------- Iteration 1-----------

Values:
---------------------------
 0.00| 5.00| 10.00| 0.00|
---------------------------
 0.00|-inf| 5.00| 2.50|
---------------------------
 0.00| 1.25| 2.50| 1.25|

------- Iteration 2-----------

Values:
---------------------------
 2.50| 5.00| 10.00| 0.00|
---------------------------
 1.25|-inf| 5.00| 2.50|
---------------------------
 0.62| 1.25| 2.50| 1.25|

------- Iteration 3-----------

Values:
---------------------------
 2.50| 5.00| 10.00| 0.00|
---------------------------
 1.25|-inf| 5.00| 2.50|
---------------------------
 0.62| 1.25| 2.50| 1.25|

policy:

---------------------

The above cell shows the value iteration for 4 iterations and in the end when the agent acts greedily to give out an optimal policy

# Stochastic Environment

In [268]:
class Grid:
    def __init__(self, width, height, start):
        self.width = width
        self.height = height
        self.row = start[0]
        self.col = start[1]
        
    
    def step(self, action):
        #A Slight variation of the deterministic environment where each action is stochastic in nature
        if self.stoch_prob < 1:
            p = np.random.random()
            if p <= self.stoch_prob:
                pass
            else:
                if action == "L" or action == "R":
                    action = np.random.choice(["U", "D"], p =[0.15, 0.05])
                    #probability of left or right = 0.8, probability of up = 0.15, down = 0.05
                elif action == "U" or action == "D":
                    action = np.random.choice(["L", "R"], p=[0.03, 0.17])
                    #probability of up or down = 0.8, probability of right = 0.17, left = 0.03

                
        if action in self.actions[(self.row, self.col)]:
            if action == "L":
                self.row -= 1
            elif action == "R":
                self.row += 1
            elif action == "U":
                self.col += 1
            elif action == "D":
                self.col -= 1
            return self.rewards.get((self.row, self.col), 0)
            
            
    def reset(self, rewards, actions, stoch_prob):
        self.actions = actions
        self.rewards = rewards
        self.stoch_prob = stoch_prob
        
    def find_next_state_reward(self, action):
        agent_pos_row = self.row
        agent_pos_col = self.col
        if action in self.actions[(agent_pos_row, agent_pos_col)]:
            if action == "U":
                if self.check_boundary_and_invalid(agent_pos_row-1, agent_pos_col):
                    agent_pos_row -= 1
                else:
                    pass
            elif action == "D":
                if self.check_boundary_and_invalid(agent_pos_row+1, agent_pos_col ):
                    agent_pos_row += 1
                else:
                    pass
            elif action == "R":
                if self.check_boundary_and_invalid(agent_pos_row, agent_pos_col+1):
                    agent_pos_col += 1
                else:
                    pass
            elif action == "L":
                if self.check_boundary_and_invalid(agent_pos_row, agent_pos_col-1):
                    agent_pos_col -= 1 
                else:
                    pass
        else:
            pass
        
        reward = self.rewards.get((agent_pos_row, agent_pos_col), 0)
        return ((agent_pos_row, agent_pos_col), reward)
    
    def check_boundary_and_invalid(self,row, col):
        if row >=0 and row <= self.height:
                if col >=0 and col <= self.width:
                    return True
        return False
        
    def non_terminal_states(self):
        return self.actions.keys()

    def set_agent_position(self, s):
        self.row = s[0]
        self.col = s[1]
    
    def trans_probs(self, action):
        # returns a transition probablity, reward, state tuple
        trans_probs = []
        state, reward = self.find_next_state_reward(action)
        trans_probs.append((self.stoch_prob, reward, state))
        stoch_prob_inv = 1 - self.stoch_prob

        if (stoch_prob_inv == 0.0):
            return trans_probs
        
        #Below lines incorporate stochasticity into the actions taken by the agent
        if action == "L" or action == "R":
            state, reward = self.find_next_state_reward("U")
            trans_probs.append((0.15, reward, state)) 
            state, reward = self.find_next_state_reward("D")
            trans_probs.append((0.05, reward, state))
        elif action == "U" or action == "D":
            state, reward = self.find_next_state_reward("L")
            trans_probs.append((0.03, reward, state))
            state, reward = self.find_next_state_reward("R")
            trans_probs.append((0.17, reward, state))
        return trans_probs
    
    def all_states(self):
        return set(self.actions.keys()) | set(self.rewards.keys())
        

In [269]:
class Agent():
    def __init__(self, grid, discount):
        self.environment = grid
        self.state_value = {}
        self.action_value = {}
        self.valid_actions = ("U","D","L","R") # (Up, Down, Left, Right)
        self.discount = discount
        
        for s in grid.all_states():
            self.state_value[s] = 0
        
    
    def find_value_action(self, s):
        best_action = "GO"
        best_value = float('-inf')
        
        self.environment.set_agent_position(s)
        legal_actions = self.environment.actions[s]
        for action in legal_actions:
            transitions = self.environment.trans_probs(action)
            expected_value = 0
            expected_reward = 0
            for (prob, immediate_reward, next_state) in transitions:
                expected_reward += prob * immediate_reward
                expected_value += prob * self.state_value[next_state]
            current_value = expected_reward + discount * expected_value
            if current_value > best_value:
                best_value = current_value
                best_action = action
        return best_action, best_value

    def find_v(self):
        states = self.environment.all_states()
        for s in self.environment.non_terminal_states():
            previous_state_value = self.state_value[s]
#             print("previous_state:",previous_state_value)
            _, new_state_value = self.find_value_action(s)
#             print("new_state:", new_state_value)
            self.state_value[s] = new_state_value
        return self.state_value

    def initialize_random_policy(self):
        policy = {}
        for s in self.environment.non_terminal_states():
            policy[s] = np.random.choice(self.valid_actions)
        return policy

    def calculate_greedy_policy(self,V):
        policy = self.initialize_random_policy()
        for s in policy.keys():
            self.environment.set_agent_position(s)
            best_action, _ = self.find_value_action(s)
            policy[s] = best_action
        return policy
    
    def value_iteration(self, iterations):
        print("Rewards:")
        print_values(self.environment.rewards, self.environment)
        
        for i in range(iterations):
            print("\n------- Iteration {}-----------\n".format(i))
            values = self.find_v()
            print("Values:")
            print_values(values, self.environment)
            
        greedy_policy = self.calculate_greedy_policy(values)
        print("\npolicy:")
        print_policy(greedy_policy, self.environment)

Since it is assumed the full knowledge of MDP for value iteration, the actions are given expilictly while creating the grid object

In [270]:
def grid(stoch_prob):
    g = Grid(3, 4, (2, 0))
    rewards = {(0, 3): 10, 
               (1, 3): -5,}
    actions = { (0, 0): ('D', 'R'), (0, 1): ('L', 'R'), (0, 2): ('L', 'D', 'R'), (1, 0): ('U', 'D'), (1,1): (),
    (1, 2): ('U', 'D', 'R'), (1,3) : ("L", "D"),  (2, 0): ('U', 'R'), (2, 1): ('L', 'R'),(2, 2): ('L', 'R', 'U'),(2, 3): ('L', 'U'),}
    g.reset(rewards, actions, stoch_prob)
    return g

In the stochastic environment, a variable for the stochasticity is introduced which makes the actions of the agents stochastic.

In [271]:
g = grid(0.8)
a = Agent(g, 0.9)

### Results of Stochastic Environment

In [272]:
a.value_iteration(5)

Rewards:
---------------------------
 0.00| 0.00| 0.00| 10.00|
---------------------------
 0.00| 0.00| 0.00|-5.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|

------- Iteration 0-----------

Values:
---------------------------
 0.00| 0.00| 8.00| 0.00|
---------------------------
 0.00|-inf| 2.35| 0.19|
---------------------------
 0.00| 0.00| 0.94|-0.36|

------- Iteration 1-----------

Values:
---------------------------
 0.00| 3.20| 8.66| 0.00|
---------------------------
 0.00|-inf| 2.66| 0.32|
---------------------------
 0.00| 0.38| 1.04|-0.32|

------- Iteration 2-----------

Values:
---------------------------
 1.28| 3.78| 8.72| 0.00|
---------------------------
 0.51|-inf| 2.70| 0.35|
---------------------------
 0.24| 0.45| 1.06|-0.31|

------- Iteration 3-----------

Values:
---------------------------
 1.62| 3.86| 8.72| 0.00|
---------------------------
 0.70|-inf| 2.71| 0.35|
---------------------------
 0.32| 0.47| 1.06|-0.31|

------- Iteration 4-----------

Va